In [0]:
from pyspark.sql.functions import when, col
import pandas as pd

df = spark.read.table("hcmut.gold.fact_vn_weather_hourly")
selected_columns = [
    "dt_date_record",
    "ds_location",
    "nr_temperature_2m",
    "nr_dew_point_2m",
    "nr_relative_humidity_2m",
    "nr_pressure_msl",
    "nr_precipitation",
    "nr_cloud_cover",
    "nr_wind_speed_10m",
    "nr_wind_direction_10m",
    "nr_sunshine_duration"
]

df = df.withColumn(
    "loc_hcm",
    when(col("ds_location") == "Ho Chi Minh City", 1).otherwise(0)
).withColumn(
    "loc_dn",
    when(col("ds_location") == "Da Nang City", 1).otherwise(0)
).withColumn(
    "loc_hn",
    when(col("ds_location") == "Ha Noi City", 1).otherwise(0)
)
selected_columns.extend(["loc_hcm", "loc_dn", "loc_hn"])

df = (
    df.select(*selected_columns)
      .dropna()
      .dropDuplicates(["dt_date_record"])
      .orderBy("dt_date_record")
)

pdf = df.toPandas()
pdf['dt_date_record'] = pd.to_datetime(pdf['dt_date_record'])
pdf = pdf.set_index('dt_date_record')

In [0]:
from sklearn.preprocessing import MinMaxScaler

numeric_cols = [
    "nr_temperature_2m",
    "nr_dew_point_2m",
    "nr_relative_humidity_2m",
    "nr_pressure_msl",
    "nr_precipitation",
    "nr_cloud_cover",
    "nr_wind_speed_10m",
    "nr_wind_direction_10m",
    "nr_sunshine_duration"
]

location_cols = ["loc_hcm", "loc_dn", "loc_hn"]
scaler = MinMaxScaler()
scaled_numeric = scaler.fit_transform(pdf[numeric_cols])

scaled_data = np.hstack([scaled_numeric, pdf[location_cols].values])

In [0]:
import numpy as np

# --- 1. ĐỊNH NGHĨA CÁC THAM SỐ CẤU HÌNH ---
# (Hãy đảm bảo các biến này khớp với mô hình của bạn)

# scaled_data = ... (mảng numpy (total_rows, 12) của bạn từ cell trước)
# pdf = ...           (Pandas DataFrame từ cell trước)

N_INPUT_HOURS = 48  # (Số giờ input, ví dụ: 48)
N_OUTPUT_HOURS = 24 # (Số giờ output, PHẢI là 24)
TARGET_COL_INDEX = 0 # (Vị trí cột 'nr_temperature_2m', là 0)

# --- 2. TÁCH TRAIN/TEST (THEO THỜI GIAN) ---
# Chúng ta phải tách trước khi tạo chuỗi để tránh rò rỉ dữ liệu.
# KHÔNG BAO GIỜ dùng train_test_split(..., shuffle=True) cho time series.

test_split_percentage = 0.2
test_split_index = int(len(scaled_data) * (1 - test_split_percentage))

train_data = scaled_data[:test_split_index]
test_data = scaled_data[test_split_index:]

# (Tùy chọn) Lưu lại index thời gian để vẽ biểu đồ
train_index = pdf.index[:test_split_index]
test_index = pdf.index[test_split_index:]

print(f"--- TÁCH DỮ LIỆU ---")
print(f"Total data shape: {scaled_data.shape}")
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

# --- 3. ĐỊNH NGHĨA HÀM TẠO CHUỖI (ĐÃ SỬA) ---
# Hàm này sẽ tạo y_train có shape (samples, 24)

def create_sequences(data, n_input, n_output, target_col_index):
    X_list = [] # Danh sách chứa các chuỗi input
    y_list = [] # Danh sách chứa các chuỗi output (24 giờ)
    
    for i in range(len(data)):
        # Tìm điểm cuối của 2 cửa sổ (input và output)
        end_x = i + n_input
        end_y = end_x + n_output
        
        # Kiểm tra xem có vượt quá giới hạn mảng không
        if end_y > len(data):
            break
            
        # Lấy X (input): shape (n_input, n_features)
        # Tức là (48, 12)
        seq_x = data[i:end_x, :]
        X_list.append(seq_x)
        
        # Lấy y (target): shape (n_output,)
        # Tức là (24,)
        # CHỈ lấy cột mục tiêu (nhiệt độ)
        seq_y = data[end_x:end_y, target_col_index]
        y_list.append(seq_y)
        
    return np.array(X_list), np.array(y_list)

# --- 4. TẠO DỮ LIỆU HUẤN LUYỆN VÀ KIỂM THỬ ---
print("\nĐang tạo chuỗi cho X_train, y_train...")
X_train, y_train = create_sequences(train_data, N_INPUT_HOURS, N_OUTPUT_HOURS, TARGET_COL_INDEX)

print("Đang tạo chuỗi cho X_test, y_test...")
X_test, y_test = create_sequences(test_data, N_INPUT_HOURS, N_OUTPUT_HOURS, TARGET_COL_INDEX)

# (Tùy chọn) Lưu lại index cho y_test (dùng để vẽ biểu đồ)
# Cần logic phức tạp hơn một chút để lấy đúng index cho y_test
# (Bỏ qua nếu bạn không cần vẽ biểu đồ)


# --- 5. (TRẢ LỜI) KIỂM TRA SHAPE ---
print("\n--- KIỂM TRA SHAPE (QUAN TRỌNG) ---")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

print("\n--- KIỂM TRA 1 MẪU ---")
print(f"Một mẫu X_train (shape {X_train[0].shape}): \n{X_train[0][:2]}...") # In 2 dòng đầu của mẫu đầu tiên
print(f"Một mẫu y_train (shape {y_train[0].shape}): \n{y_train[0]}...") # In 24 giá trị của mẫu đầu tiên

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(32),
    Dense(16, activation='relu'),
    Dense(24)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

history = model.fit(X_train, y_train, validation_split=0.1, epochs=20, batch_size=32)


In [0]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

n_numeric = len(numeric_cols) # (Đây là 9)

# 1. Lấy dự đoán từ model
# Shape của X_test là (10229, 48, 12)
# Shape của y_pred_scaled sẽ là (10229, 24)
y_pred_scaled = model.predict(X_test)

# 2. Làm dẹt (flatten) CẢ HAI mảng dự đoán và mảng thực tế
# Shape của cả hai sẽ trở thành (10229 * 24,) = (245496,)
y_pred_flat = y_pred_scaled.reshape(-1)
y_test_flat = y_test.reshape(-1) # <--- y_test của bạn đã có shape (10229, 24)

# 3. Tạo các "dummy array" LỚN tương ứng
# Chúng phải có shape (245496, 9)
dummy_pred = np.zeros((len(y_pred_flat), n_numeric))
dummy_real = np.zeros((len(y_test_flat), n_numeric))

# 4. Đặt dữ liệu đã làm dẹt vào cột 0 (cột nhiệt độ)
dummy_pred[:, 0] = y_pred_flat
dummy_real[:, 0] = y_test_flat

# 5. Bây giờ mới Inverse Transform
# Cả hai mảng đầu ra sẽ có shape (245496,)
y_pred_unscaled = scaler.inverse_transform(dummy_pred)[:, 0]
y_test_unscaled = scaler.inverse_transform(dummy_real)[:, 0]

# 6. Tính toán Metrics trên toàn bộ dữ liệu
print("--- Đánh giá trên toàn bộ 24 giờ dự đoán ---")

rmse = np.sqrt(mean_squared_error(y_test_unscaled, y_pred_unscaled))
r2 = r2_score(y_test_unscaled, y_pred_unscaled)
mae = mean_absolute_error(y_test_unscaled, y_pred_unscaled)

# Xử lý trường hợp chia cho 0 (mặc dù hiếm với nhiệt độ)
mask = y_test_unscaled != 0
mape = np.mean(np.abs((y_test_unscaled[mask] - y_pred_unscaled[mask]) / y_test_unscaled[mask])) * 100

print(f"RMSE: {rmse:.3f} °C")
print(f"R²  : {r2:.2f}")
print(f"MAE : {mae:.3f} °C")
print(f"MAPE: {mape:.2f}%")

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from zoneinfo import ZoneInfo

location_name = "Ho Chi Minh City"
location_map = {
    "Ho Chi Minh City": [1,0,0],
    "Da Nang City": [0,1,0],
    "Ha Noi City": [0,0,1],
}
location_onehot = np.array(location_map[location_name])

pdf_loc = pdf[pdf['ds_location'] == location_name].copy()
if len(pdf_loc) < 48:
    raise ValueError(f"Not enough data for {location_name} to forecast 48h")

last_48_indexed = pdf_loc.index[-48:]
last_48 = scaled_data[-48:].copy() 
last_48[:, -3:] = location_onehot

def forecast_next_24_hours(model, last_seq, location_onehot, n_steps=24):
    seq = last_seq.copy()
    predictions = []
    for _ in range(n_steps):
        X = seq.reshape(1, seq.shape[0], seq.shape[1])
        pred_scaled = model.predict(X)[0][0]
        predictions.append(pred_scaled)

        fake_record = seq[-1].copy()
        fake_record[0] = pred_scaled
        fake_record[-3:] = location_onehot
        seq = np.vstack([seq[1:], fake_record])
    return np.array(predictions)

pred_scaled_24 = forecast_next_24_hours(model, last_48, location_onehot)
dummy_forecast = np.zeros((len(pred_scaled_24), len(numeric_cols)))
dummy_forecast[:, 0] = pred_scaled_24
pred_24_real = scaler.inverse_transform(dummy_forecast)[:, 0]

last_48_index_test = pdf_loc.index[-48:]
last_48_real = y_test_real[-48:]
last_48_pred = y_pred[-48:]

vn_now = last_48_index_test[-1]
forecast_index = [vn_now + pd.Timedelta(hours=i+1) for i in range(24)]

plt.figure(figsize=(16,5))
plt.plot(last_48_index_test, last_48_real, marker='o', label="Real (Last 48h Test)")
plt.plot(last_48_index_test, last_48_pred, marker='x', label="Predicted (Last 48h Test)")

plt.plot(forecast_index, pred_24_real, marker='s', linestyle='--', label="Forecast Next 24h")

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0,24,6))) 
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m %H:%M'))

plt.xticks(rotation=45)
plt.xlabel("Time")
plt.ylabel("Temperature (°C)")
plt.title(f"48h Test and 24h Forecast ({location_name})")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [0]:
# --- BƯỚC 7 (GỘP VÀ CLEAN): LOGGING VỚI UNITY CATALOG & SIGNATURE ---

import mlflow
import mlflow.keras
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np

# --- 7.1. Lấy các biến đã huấn luyện từ các cell trước ---
# (Đảm bảo các biến này đã tồn tại sau khi huấn luyện)
# N_INPUT_HOURS, N_OUTPUT_HOURS, numeric_cols, location_cols, 
# FEATURE_COLUMNS, TARGET_COL_INDEX, model, scaler, pdf, history

try:
    # Gán vào tên biến chuẩn
    trained_model = model
    trained_scaler = scaler
    
    print("Đã xác định các biến huấn luyện (model, scaler, pdf, history, etc.).")
    print(f"Input hours: {N_INPUT_HOURS}, Output hours: {N_OUTPUT_HOURS}")
    print(f"Target column index: {TARGET_COL_INDEX}")

except NameError as e:
    print(f"LỖI: Không tìm thấy biến cần thiết. Bạn đã chạy cell huấn luyện chưa?")
    print(f"Chi tiết: {e}")
    # dbutils.notebook.exit("Biến huấn luyện không tồn tại")


# --- 7.2. Định nghĩa Lớp Wrapper (xử lý "scale một phần") ---

class LSTMWeatherWrapper(mlflow.pyfunc.PythonModel):

    def __init__(self, model, scaler, n_input_hours, n_output_hours, 
                 numeric_cols, location_cols, target_col_index):
        self.model = model
        self.scaler = scaler
        self.n_input = n_input_hours
        self.n_output = n_output_hours
        
        # Lưu lại danh sách các cột
        self.numeric_cols = numeric_cols
        self.location_cols = location_cols
        self.feature_columns = numeric_cols + location_cols
        
        self.n_features = len(self.feature_columns)
        self.n_numeric_features = len(numeric_cols)
        self.target_idx = target_col_index

    def _preprocess(self, model_input_df):
        # Đảm bảo DataFrame có đúng các cột
        processed_df = model_input_df[self.feature_columns]
        
        # 1. Tách riêng 2 nhóm cột
        numeric_data = processed_df[self.numeric_cols]
        location_data = processed_df[self.location_cols].values
        
        # 2. CHỈ scale các cột numeric
        scaled_numeric = self.scaler.transform(numeric_data)
        
        # 3. Kết hợp lại (giống hệt code training)
        scaled_data = np.hstack([scaled_numeric, location_data])
        
        # 4. Reshape cho LSTM [samples, timesteps, features]
        input_sequence = scaled_data.reshape(1, self.n_input, self.n_features)
        return input_sequence

    def _postprocess(self, prediction_scaled):
        # prediction_scaled có shape (1, n_output)
        
        # Tạo một mảng "giả" (dummy) CHỈ có shape (n_output, n_numeric_features)
        dummy_array = np.zeros((self.n_output, self.n_numeric_features))
        
        # Đặt kết quả dự đoán (đã chuẩn hóa) vào đúng cột mục tiêu
        dummy_array[:, self.target_idx] = prediction_scaled
        
        # 3. Inverse transform (Biến đổi ngược)
        prediction_unscaled = self.scaler.inverse_transform(dummy_array)
        
        # 4. Chỉ lấy cột mục tiêu (nhiệt độ) đã được unscale
        final_forecast = prediction_unscaled[:, self.target_idx]
        
        return final_forecast

    def predict(self, context, model_input):
        # 'model_input' là một Pandas DataFrame của (N_INPUT_HOURS, N_FEATURES)
        
        if model_input.shape[0] != self.n_input:
            raise ValueError(f"Input data must have exactly {self.n_input} rows (timesteps).")
            
        # Bước 1-4: Tiền xử lý (Tách, Scale, Hstack, Reshape)
        input_sequence = self._preprocess(model_input)
        
        # Bước 5: Dự đoán
        prediction_scaled = self.model.predict(input_sequence)
        
        # Bước 6-7: Hậu xử lý (Inverse Transform)
        final_forecast = self._postprocess(prediction_scaled.flatten())
        
        return final_forecast.tolist() # Trả về list 24 con số

print("Đã định nghĩa lớp 'LSTMWeatherWrapper'.")

# --- 7.3. Tạo Signature và Logging vào Unity Catalog ---

# 1. TẠO INPUT VÀ OUTPUT MẪU ĐỂ SUY RA SIGNATURE
print("Đang tạo đối tượng wrapper...")
pyfunc_wrapper = LSTMWeatherWrapper(
    model=trained_model,
    scaler=trained_scaler,
    n_input_hours=N_INPUT_HOURS,
    n_output_hours=N_OUTPUT_HOURS,
    numeric_cols=numeric_cols,
    location_cols=location_cols,
    target_col_index=TARGET_COL_INDEX
)

print("Đang chuẩn bị dữ liệu mẫu cho signature...")
try:
    # Lấy một mẫu input (ví dụ: N_INPUT_HOURS giờ đầu tiên từ 'pdf')
    input_example = pdf[FEATURE_COLUMNS].iloc[0:N_INPUT_HOURS]
    
    if input_example.shape[0] != N_INPUT_HOURS:
        raise ValueError(f"Dữ liệu 'pdf' không đủ {N_INPUT_HOURS} dòng để làm mẫu.")

except NameError:
    print("LỖI: Không tìm thấy 'pdf'. Bạn phải chạy lại cell training để có 'pdf'.")
    # dbutils.notebook.exit("Cần 'pdf' để tạo signature")
except Exception as e:
    print(f"LỖI khi lấy input_example: {e}")
    # dbutils.notebook.exit("Lỗi signature")

print(f"Đang chạy dự đoán mẫu trên {input_example.shape[0]} dòng...")
# Chạy dự đoán mẫu để lấy output
output_example = pyfunc_wrapper.predict(context=None, model_input=input_example)

# 2. SUY RA SIGNATURE
print("Đang suy ra signature từ input và output mẫu...")
signature = infer_signature(input_example, output_example)
print("--- Đã tạo signature thành công ---")


# 3. LOG VÀ ĐĂNG KÝ MÔ HÌNH (VỚI SIGNATURE)

# !!! QUAN TRỌNG: Sửa lại <CATALOG_NAME> và <SCHEMA_NAME> của bạn
UC_CATALOG_NAME = "hcmut"  # !!! THAY THẾ: Bằng Catalog của bạn
UC_SCHEMA_NAME = "gold"    # !!! THAY THẾ: Bằng Schema của bạn (ví dụ: 'gold' hoặc 'ml')
MODEL_NAME = "WeatherForecast_LSTM_MultiCity"
model_registry_name = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.{MODEL_NAME}"

# Set Model Registry về UC
mlflow.set_registry_uri("databricks-uc")

# Đặt experiment
experiment_name = f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/LSTM_Weather_Forecast"
mlflow.set_experiment(experiment_name)

print(f"Bắt đầu logging vào Experiment: {experiment_name}")
print(f"Đăng ký mô hình UC với tên: {model_registry_name}")

with mlflow.start_run(run_name="LSTM_24h_Forecast_UC_Run") as run:
    
    # Log các tham số
    mlflow.log_param("n_input_hours", N_INPUT_HOURS)
    mlflow.log_param("n_output_hours", N_OUTPUT_HOURS)
    mlflow.log_param("feature_columns_count", len(FEATURE_COLUMNS))

    # (Tùy chọn) Log các chỉ số
    if 'history' in locals():
        mlflow.log_metric("final_val_loss", history.history['val_loss'][-1])
        mlflow.log_metric("final_train_loss", history.history['loss'][-1])

    # 4. LOG MÔ HÌNH (Sử dụng đối tượng wrapper đã tạo ở trên)
    print("Đang log mô hình với signature...")
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=pyfunc_wrapper,
        registered_model_name=model_registry_name,
        signature=signature  # <-- THÊM SIGNATURE VÀO ĐÂY
    )
    
    run_id = run.info.run_id
    print(f"--- HOÀN TẤT LOGGING (với Signature) ---")
    print(f"Run ID: {run_id}")
    print(f"Đã đăng ký phiên bản mới cho mô hình: '{model_registry_name}'")

In [0]:
# --- BƯỚC 8 (ĐÃ SỬA): PIPELINE DỰ BÁO TỰ ĐỘNG ---
# (Tương thích với Unity Catalog Aliases)

import mlflow
import pandas as pd
from pyspark.sql.functions import col, when
import datetime

# --- 8.1. Các tham số cấu hình ---

# !!! QUAN TRỌNG: Sửa lại <CATALOG_NAME> và <SCHEMA_NAME> của bạn
UC_CATALOG_NAME = "hcmut"
UC_SCHEMA_NAME = "gold"
MODEL_NAME = "WeatherForecast_LSTM_MultiCity"
MODEL_REGISTRY_NAME = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.{MODEL_NAME}"

# --- (SỬA 1) ---
# MODEL_STAGE = "Production" # <-- BỎ DÒNG NÀY
MODEL_ALIAS = "prod" # <-- THÊM DÒNG NÀY (Phải khớp với alias bạn đặt ở Bước 1)
# ---------------

# Tên bảng Delta Lake (nguồn)
SOURCE_TABLE = "hcmut.gold.fact_vn_weather_hourly"
# Tên bảng Delta Lake (đích) để lưu kết quả
TARGET_TABLE = "hcmut.gold.lstm_weather_24h"

# Các thành phố cần dự báo
CITIES_TO_FORECAST = ["Ho Chi Minh City", "Da Nang City", "Ha Noi City"]

# Các thông số của mô hình (PHẢI GIỐNG HỆT KHI HUẤN LUYỆN)
N_INPUT_HOURS = 48 
N_OUTPUT_HOURS = 24

numeric_cols = [
    "nr_temperature_2m", "nr_dew_point_2m", "nr_relative_humidity_2m",
    "nr_pressure_msl", "nr_precipitation", "nr_cloud_cover",
    "nr_wind_speed_10m", "nr_wind_direction_10m", "nr_sunshine_duration"
]
location_cols = ["loc_hcm", "loc_dn", "loc_hn"]
FEATURE_COLUMNS = numeric_cols + location_cols
BASE_COLUMNS_TO_SELECT = ["dt_date_record", "ds_location"] + numeric_cols

# --- 8.2. Tải Mô hình "All-in-One" từ Registry ---

# Set registry về Unity Catalog
mlflow.set_registry_uri("databricks-uc")

# --- (SỬA 2) ---
print(f"Đang tải mô hình '{MODEL_REGISTRY_NAME}' với bí danh (alias) '@{MODEL_ALIAS}'...")
# Cú pháp mới: models:/<model_name>@<alias>
model_uri = f"models:/{MODEL_REGISTRY_NAME}@{MODEL_ALIAS}"
# ---------------

try:
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    print("--- Tải mô hình thành công ---")
except Exception as e:
    # --- (SỬA 3) ---
    print(f"LỖI: Không thể tải mô hình. Bạn đã đặt alias '@{MODEL_ALIAS}' cho phiên bản mới nhất chưa?")
    print(f"Chi tiết lỗi: {e}")
    dbutils.notebook.exit(f"Model loading failed. Check alias '@{MODEL_ALIAS}'.") # <-- KÍCH HOẠT DÒNG NÀY
    # ---------------

# --- 8.3. Lấy Dữ liệu, Xử lý và Dự báo (Lặp qua từng thành phố) ---

all_forecasts = [] 

for city_name in CITIES_TO_FORECAST:
    print(f"\n--- Đang xử lý cho: {city_name} ---")
    
    try:
        # Lấy N_INPUT_HOURS (ví dụ: 48) bản ghi mới nhất
        print(f"Đang lấy {N_INPUT_HOURS} giờ dữ liệu mới nhất...")
        latest_data_df = (
            spark.read.table(SOURCE_TABLE)
            .select(*BASE_COLUMNS_TO_SELECT)
            .withColumn("loc_hcm", when(col("ds_location") == "Ho Chi Minh City", 1).otherwise(0))
            .withColumn("loc_dn", when(col("ds_location") == "Da Nang City", 1).otherwise(0))
            .withColumn("loc_hn", when(col("ds_location") == "Ha Noi City", 1).otherwise(0))
            .filter(col("ds_location") == city_name)
            .select(["dt_date_record"] + FEATURE_COLUMNS) 
            .orderBy(col("dt_date_record").desc())
            .limit(N_INPUT_HOURS) 
            .orderBy(col("dt_date_record").asc()) 
        ).toPandas() 

        if latest_data_df.shape[0] != N_INPUT_HOURS:
            print(f"LỖI: Không tìm thấy đủ {N_INPUT_HOURS} giờ dữ liệu cho {city_name}. Tìm thấy {latest_data_df.shape[0]} giờ. Bỏ qua...")
            continue 

        print(f"Dữ liệu đầu vào (từ {latest_data_df['dt_date_record'].min()} đến {latest_data_df['dt_date_record'].max()})")

        # --- 8.4. Thực hiện Dự báo ---
        print("Đang thực hiện dự báo...")
    
        forecast_values = loaded_model.predict(latest_data_df[FEATURE_COLUMNS])

        print("--- Dự báo thành công ---")

        # --- 8.5. Xử lý Kết quả (Cho thành phố này) ---
        last_known_time = latest_data_df['dt_date_record'].max()
        forecast_index = [
            last_known_time + pd.Timedelta(hours=i + 1) for i in range(N_OUTPUT_HOURS)
        ]

        results_df = pd.DataFrame({
            'dt_forecast_time': forecast_index,
            'nr_predicted_temperature': forecast_values,
            'ds_location': city_name,
            'dt_model_run_time': datetime.datetime.now(datetime.timezone.utc),
            'ds_model_version_uri': model_uri # Lưu lại URI của mô hình đã dự báo
        })
        
        all_forecasts.append(results_df)

    except Exception as e:
        print(f"LỖI trong quá trình xử lý cho {city_name}: {e}")
        # (Nếu có lỗi ở đây, chúng ta vẫn tiếp tục với thành phố tiếp theo)

# --- 8.6. Lưu Tất cả Kết quả vào Delta Lake ---

if not all_forecasts:
    print("\nKhông có kết quả dự báo nào được tạo ra. Kết thúc.")
else:
    print(f"\n--- TỔNG HỢP KẾT QUẢ DỰ BÁO ({len(all_forecasts)} thành phố) ---")
    final_results_df = pd.concat(all_forecasts)
    display(final_results_df)

    print(f"Đang lưu tất cả kết quả vào bảng {TARGET_TABLE}...")
    
    try:
        results_spark_df = spark.createDataFrame(final_results_df)
        (results_spark_df.write
            .format("delta")
            .mode("append")
            .option("mergeSchema", "true") 
            .saveAsTable(TARGET_TABLE)
        )
        print("--- LƯU KẾT QUẢ VÀO DELTA LAKE THÀNH CÔNG ---")
    except Exception as e:
        print(f"LỖI khi lưu vào Delta Lake: {e}")

In [0]:
model.summary()